In [7]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import re
%matplotlib inline   

## Change elements below

In [8]:
expts=['071018_TL']
wells=[['C03','C04','C05','C06','C07','C08']]
sites=[9]
conditions={'Condition1': ['071018_TL_C03'],
           'Condition2': ['071018_TL_C04'],
           'Condition3': ['071018_TL_C05'],
           'Condition4': ['071018_TL_C06'],
           'Condition5': ['071018_TL_C07'],
           'Condition6': ['071018_TL_C08']}



framemax=28      

inpath='Inpath_here'

##TL_Cutoffs
area90=3997.338161
perim90=546.636073
circ95=0.851634
round95=0.884005
solidity95= 0.942082
gfp5=544.586362

##Ii cutoffs
#area90=4332.853226
#perim90=676.561390
#circ95=0.834039
#round95=0.857243 
#solidity95=0.934887
#gfp5=479.339356





# Get mean Fluor intensity for each cell

In [9]:
def getGFPmean(e,w,s,cell):
    
    #parse name to get path to input file:
    cellnum=re.search(r"Cell_([0-9]*)", cell)
    cellnum=cellnum.group(1)
    sitePath=inpath+'/well'+w+'/Adapt_v1.189/'+e+'_'+w+'_s'+str(s+1)+'_w2_Output/'
    #Ii#sitePath=inpath+'/well'+w+'/Adapt_v1.189/'+e+'_'+w+'_w2_'+'s'+str(s+1)+'_Output/'
    #NovDec#sitePath=inpath+e+'/'+e+'_w2/well'+w+'/Adapt_v1.189/'+e+'_w2_'+w+'_s'+str(s+1)+'_Output/'
    fname=sitePath+str(cellnum)+'_Output/MeanFluorescenceIntensity.csv' 
    
    #get mean of first frame fluor intensity
    filedf=pd.read_csv(fname, index_col=0)
    firstframe=filedf.iloc[:,0]
    gfpmean=firstframe.mean()     
    
    return gfpmean
    
    

In [10]:
def statsFiltering(df):    #cell stat dataframe
    
    test=1
    #for i in range(len(df)):   #requires that area and perim be high in same frame
    #    if (df.Area.iloc[i] > area90).all() and (df.Perim.iloc[i] > perim90).all():    #all frames
    #        test=0
    
    for i in range(len(df)):   #requires that area and perim be high ANY same frame
        if (df.Area.iloc[i] > area90).all() or (df.Perim.iloc[i] > perim90).all():    #all frames
            test=0
            
    if (df.Circ.iloc[0] > circ95).all():    #first frame
        if (df.Round.iloc[0] > round95).all():    #first frame
            if (df.Solidity.iloc[0] > solidity95).all():    #first frame
                test=0
    
    return test    #true if the cell passes
                        

## Get stats dataframe for each cell and remove cells not present in all frames

In [11]:
##Get cell names and indices in file

def wellstats(expt,well,sitenum):
    arealist=[]
    perimlist=[]
    circlist=[]
    ARlist=[]
    roundlist=[]
    soliditylist=[]
    for s in range(sitenum):
        inputfilename=inpath+'/well'+w+'/Adapt_v1.189/'+e+'_'+w+'_s'+str(s+1)+'_w2_Output/morphology.txt'
        #Ii#inputfilename=inpath+'/well'+well+'/Adapt_v1.189/'+expt+'_'+well+'_w2_'+'s'+str(s+1)+'_Output/morphology.txt'
        #Nov-Dec#inputfilename=inpath+expt+'/'+expt+'_w2/well'+well+'/Adapt_v1.189/'+expt+'_w2_'+well+'_s'+str(s+1)+'_Output/morphology.txt'

        if os.path.getsize(inputfilename) > 0:     #to deal with empty morph files.
            all_data = pd.read_csv(inputfilename, sep='\t')
            all_data=all_data.rename(columns={' ':'Cell'})
            cellindices=[]
            celllabels=[]
            for i in range(len(all_data.index)):
                if 'Cell_' in all_data.Cell.iloc[i]:
                    cellindices.append(i)
                    celllabels.append(all_data.Cell.iloc[i])

            ##Get frame rows for each cell and make dataframe for each
            ##Name each dataframe well_site_cell and put into dictionary

            last=len(all_data.index)
            cellindices=cellindices+[last]
            base=cellindices[len(cellindices)-2]
            cellindices[:0]=[0]

            celldict={}

            for n, c in enumerate(cellindices):
                if n==0:
                    dfslice=all_data.iloc[(base+cellindices[n+1]-cellindices[n]+1):(base+cellindices[n+2]-cellindices[n+1]),:]
                    name=expt+'_'+well+'_s'+str(s+1)+'_'+celllabels[n]
                    celldict[name]=dfslice
                elif n<len(cellindices)-2:
                    dfslice=all_data.iloc[(base+cellindices[n+1]-cellindices[n]):(base+cellindices[n+2]-cellindices[n+1]),:]
                    name=expt+'_'+well+'_s'+str(s+1)+'_'+celllabels[n]
                    celldict[name]=dfslice


            ##Go through cell dictionary and remove elements with less than framemax number of frames

            fullcells={}

            for name, df in celldict.iteritems():
                if len(df)>=framemax:
                    df=df.rename(columns={' ':'Cell', 'Perim.':'Perim', 'Circ.':'Circ', })
                    fullcells[name]=df
                
                
                ###Nov-Dec
                #if len(df)==framemax:
                #    df=df.rename(columns={' ':'Cell', 'Perim.':'Perim', 'Circ.':'Circ', })
                #    fullcells[name]=df

            #Filter each cell on cutoffs
            
            #fullcellsDF=pd.concat(fullcells)
            #fullcellsDF.to_csv('/Users/glennafoight/Dropbox (Personal)/Dimerizer/Microscopy/Nov2018_MembAnalysis/TL_repC_C01.csv')

            passingcells={}
            for name, df in fullcells.iteritems():
                juststats=df.iloc[:,1:]
                juststats=juststats.reset_index(drop=True)
                ##get GFP mean fluorescence and only continue for cells passing GFP cutoff
                gfpmean=getGFPmean(expt,well,s,name)
                
                if gfpmean > gfp5:
                    test=statsFiltering(juststats)
                    if test: #If the cell passes the GFP and Stats cutoffs, then include
                        passingcells[name]=juststats    #dictionary with keys=cell names, values=all stats df
                        

            #Make stat lists for all cells in a site:
            for n, cdf in passingcells.iteritems():

                areaS=pd.Series(cdf['Area'], name=n)  
                arealist.append(areaS)
        
                perimS=pd.Series(cdf['Perim'], name=n)  
                perimlist.append(perimS)
        
                circS=pd.Series(cdf['Circ'], name=n)  
                circlist.append(circS)
            
                ARS=pd.Series(cdf['AR'], name=n)  
                ARlist.append(ARS)
        
                roundS=pd.Series(cdf['Round'], name=n)  
                roundlist.append(roundS)
        
                solidityS=pd.Series(cdf['Solidity'], name=n)  
                soliditylist.append(solidityS)
                
    areadf=pd.concat(arealist, axis=1)    #concat all cells in each site
    perimdf=pd.concat(perimlist, axis=1)    #concat all cells in each site
    circdf=pd.concat(circlist, axis=1)    #concat all cells in each site
    ARdf=pd.concat(ARlist, axis=1)    #concat all cells in each site
    rounddf=pd.concat(roundlist, axis=1)    #concat all cells in each site
    soliditydf=pd.concat(soliditylist, axis=1)    #concat all cells in each site
                
                
    wellstatlist=[areadf,perimdf,circdf,ARdf,rounddf,soliditydf]    
    return wellstatlist
        

## Main

In [12]:
##Process morphology.txt files and get stat means for each cell

totaldict={}
for i, e in enumerate(expts):
    wellstatlist=[]
    for w in wells[i]:
        wellstatlist=wellstats(e,w,sites[i])
        keyname=e+'_'+w
        totaldict[keyname]=wellstatlist
        

stats=['Area','Perim','Circ','AR','Round','Solidity']    

for k, v in conditions.iteritems():

    for s in range(6):
        outfilename='/Users/glennafoight/Dropbox (Personal)/Dimerizer/Microscopy/071018_3T3_Reprocessing/TL/'+k+'_'+stats[s]+'.csv'
        replist=[]
        for rep in v:
            replist.append(totaldict[rep][s])
        conditiondf=pd.concat(replist, axis=1)
        conditiondf.to_csv(outfilename)     #Outputs csvs for each stat for each condition


        
        